## MongoDB 문제 풀어보기

[notion link](https://www.notion.so/MongoDB-845d3b14f8c84ace9044f4fd783ace7c?pvs=4)

- **복습할 개념 체크 리스트**
  - Python 함수
  - `find`
  - `sort`
  - `update_many`
  - `limit`
  - `aggregate`


## import, db 객체 초기화, faker 객체 초기화


In [87]:
from pymongo import MongoClient
from faker import Faker
from datetime import datetime
import random

ACTIONS = ["click", "view", "purchase"]
fake = Faker()

client = MongoClient("mongodb://localhost:27017/")
db = client.test_database

## 함수 정의부


In [88]:
def generate_sample_book():
    return {
        "title": fake.sentence(nb_words=3),
        "author": fake.word(
            ext_word_list=["봉준호", "James Cameron", "Christopher Nolan", "Robert Arnold"]
        ),
        "year": random.randint(1900, 2024),
        "genre": fake.word(ext_word_list=["comic", "fantasy", "thriller", "horror"]),
    }


def generate_sample_movie():
    return {
        "title": fake.sentence(nb_words=3),
        "director": fake.word(
            ext_word_list=["봉준호", "James Cameron", "Christopher Nolan", "Robert Arnold"]
        ),
        "year": random.randint(1900, 2024),
        "rating": random.uniform(1, 10),
        "genre": fake.word(ext_word_list=["comic", "fantasy", "thriller", "horror"]),
    }


def generate_user_action():
    return {
        "user_id": random.randint(1, 10),
        "action": random.choice(ACTIONS),
        "timestamp": fake.date_time_between(),
    }


def remove_all_data(db):
    for collection_name in db.list_collection_names():
        db[collection_name].drop()


def insert_data(db):
    # 책 데이터 삽입
    db["books"].insert_many([generate_sample_book() for _ in range(50)])

    # 영화 데이터 삽입
    db["movies"].insert_many([generate_sample_movie() for _ in range(50)])

    # 사용자 행동 데이터 삽입
    db["user_actions"].insert_many([generate_user_action() for _ in range(50)])

    print("Data inserted successfully")


def find_books_by_genre(db, genre):
    books_collection = db.books
    query = {"genre": genre}
    projection = {
        "_id": 0,
        "title": 1,
        "author": 1,
        "genre": 1,
    }  # 1 은 포함할 필드, 0 은 제외할 필드

    books = books_collection.find(query, projection)
    for book in books:
        print(book)


def find_recent_actions_by_user(db, user_id, limit=5):
    """
    사용자의 최근 행동 5개를 시간 순으로 정렬하여 보여주세요
    """
    _match = {"$match": {"user_id": user_id}}
    _limit = {"$limit": limit}
    # TODO - sorting
    pipeline = [_match, _limit]
    result = db["user_actions"].aggregate(pipeline)
    # TODO - find().sort().limit() 체이닝이 가능함!

    for action in result:
        print(action)


def update_user_actions_before_date(db, user_id, date, old_action, new_action):
    """
    UPDATE

    user_id의 date 이전의 old_action을 new_action으로 변경하는 함수.
    """
    query = {
        "user_id": user_id,
        "timestamp": {"$lt": date},
        "action": old_action,
    }  # user_id의 timestamp가 date 이전
    _set = {"$set": {"action": new_action}}

    db["user_actions"].update_many(query, _set)

## 데이터 초기화 및 삽입코드

데이터 초기화를 하는 이유는 매번 새로운 데이터를 얻기 위해서입니다.


In [89]:
remove_all_data(db)
insert_data(db)

Data inserted successfully


## [문제 1: 특정 장르의 책 찾기]

- **문제 설명**:
  사용자는 "fantasy" 장르의 모든 책을 찾고 싶어합니다.
- **쿼리 작성 목표**:
  "fantasy" 장르에 해당하는 모든 책의 제목과 저자를 찾는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.

**[find](https://www.mongodb.com/docs/manual/tutorial/query-documents/)**

```python
find(filter, projection, skip, limit, ...)
```


In [90]:
find_books_by_genre(db, "fantasy")

{'title': 'Project team.', 'author': 'Christopher Nolan', 'genre': 'fantasy'}
{'title': 'Share rate beautiful.', 'author': 'James Cameron', 'genre': 'fantasy'}
{'title': 'Its.', 'author': 'James Cameron', 'genre': 'fantasy'}
{'title': 'Minute suggest much long.', 'author': 'Robert Arnold', 'genre': 'fantasy'}
{'title': 'Represent point.', 'author': 'Christopher Nolan', 'genre': 'fantasy'}
{'title': 'Majority as.', 'author': 'James Cameron', 'genre': 'fantasy'}
{'title': 'Network.', 'author': 'Robert Arnold', 'genre': 'fantasy'}
{'title': 'In statement.', 'author': '봉준호', 'genre': 'fantasy'}
{'title': 'Behind partner subject.', 'author': 'Christopher Nolan', 'genre': 'fantasy'}
{'title': 'Hair.', 'author': 'Robert Arnold', 'genre': 'fantasy'}
{'title': 'Plant stay.', 'author': 'James Cameron', 'genre': 'fantasy'}
{'title': 'With morning.', 'author': 'Robert Arnold', 'genre': 'fantasy'}


## [문제 2: 감독별 평균 영화 평점 계산]

- **문제 설명**:
  각 영화 감독별로 그들의 영화 평점의 평균을 계산하고 싶습니다. 이를 통해 어떤 감독이 가장 높은 평균 평점을 가지고 있는지 알아볼 수 있습니다.
- **쿼리 작성 목표**:
  모든 영화 감독의 영화 평점 평균을 계산하고, 평균 평점이 높은 순으로 정렬하는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.

**[Aggregation Pipelines](https://www.mongodb.com/docs/manual/core/aggregation-pipeline/#std-label-aggregation-pipeline)**

- aggregation pipeline은 하나 이상의 stage로 구성되어 있습니다:
- 각 stage당 필터, 그룹, 집계 연산 중 하나를 수행합니다.
- aggregation pipeline은 도큐먼트의 그룹을 리턴합니다. 예를 들어, 총합, 평균, 최대, 최소를 구할 수 있습니다.

**[group](https://www.mongodb.com/docs/manual/reference/operator/aggregation/group/)**

`$group` 스테이지는 명시한 그룹 key 별로 하나의 도큐먼트를 생성합니다. 이때 key는 필드 (혹은 필드 집합)에 해당합니다. 그룹 결과는 `_id` 필드에 적용되어야만 합니다 (required)


In [91]:
group = {"$group": {"_id": "$director", "average_rating": {"$avg": "$rating"}}}
sort = {"$sort": {"average_rating": -1}}

pipeline = [group, sort]

for e in db.movies.aggregate(pipeline):
    print(e)

{'_id': 'Christopher Nolan', 'average_rating': 6.0988796159243925}
{'_id': 'James Cameron', 'average_rating': 5.334495114472351}
{'_id': '봉준호', 'average_rating': 5.177841512443577}
{'_id': 'Robert Arnold', 'average_rating': 4.362250770326775}


## [문제 3: 특정 사용자의 최근 행동 조회]

- **문제 설명**:
  특정 사용자의 최근 행동 로그를 조회하고자 합니다. 이 때, 최신 순으로 정렬하여 최근 5개의 행동만을 보고 싶습니다.
- **쿼리 작성 목표**:
  사용자 ID가 1인 사용자의 최근 행동 5개를 시간 순으로 정렬하여 조회하는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.

**[match](https://www.mongodb.com/docs/manual/reference/operator/aggregation/match/#mongodb-pipeline-pipe.-match)**

aggregate의 탐색범위를 줄이는 데 사용됩니다. query문과 동일한 문법을 가지고 있습니다.

예시코드:

```javascript
db.articles.aggregate([{ $match: { author: "dave" } }]);
```


In [84]:
find_recent_actions_by_user(db, 1, 5)

{'_id': ObjectId('66308c97770dc240bc3ff583'), 'user_id': 1, 'action': 'purchase', 'timestamp': datetime.datetime(2012, 5, 8, 19, 1, 32, 83000)}
{'_id': ObjectId('66308c97770dc240bc3ff591'), 'user_id': 1, 'action': 'view', 'timestamp': datetime.datetime(2001, 3, 10, 12, 23, 56, 700000)}
{'_id': ObjectId('66308c97770dc240bc3ff597'), 'user_id': 1, 'action': 'purchase', 'timestamp': datetime.datetime(2019, 1, 11, 17, 36, 26, 592000)}
{'_id': ObjectId('66308c97770dc240bc3ff59e'), 'user_id': 1, 'action': 'purchase', 'timestamp': datetime.datetime(2022, 10, 26, 21, 6, 52, 228000)}
{'_id': ObjectId('66308c97770dc240bc3ff5a0'), 'user_id': 1, 'action': 'click', 'timestamp': datetime.datetime(2006, 10, 3, 1, 37, 32, 602000)}


## [문제 4: 출판 연도별 책의 수 계산]

- **문제 설명** :
  데이터베이스에 저장된 책 데이터를 이용하여 각 출판 연도별로 출판된 책의 수를 계산하고자 합니다. 이 데이터는 시간에 따른 출판 트렌드를 분석하는 데 사용될 수 있습니다.
- **쿼리 작성 목표** :
  각 출판 연도별로 출판된 책의 수를 계산하고, 출판된 책의 수가 많은 순서대로 정렬하는 MongoDB 쿼리를 함수로 만들어 문제를 해결해 봅니다.


In [92]:
group = {"$group": {"_id": "$year", "published_count": {"$count": {}}}}
sort = {"$sort": {"_id": 1}}

for e in db["books"].aggregate([group, sort]):
    print(e)

{'_id': 1900, 'published_count': 2}
{'_id': 1901, 'published_count': 1}
{'_id': 1903, 'published_count': 2}
{'_id': 1905, 'published_count': 1}
{'_id': 1907, 'published_count': 1}
{'_id': 1908, 'published_count': 2}
{'_id': 1910, 'published_count': 1}
{'_id': 1911, 'published_count': 1}
{'_id': 1920, 'published_count': 1}
{'_id': 1923, 'published_count': 2}
{'_id': 1926, 'published_count': 1}
{'_id': 1932, 'published_count': 1}
{'_id': 1934, 'published_count': 1}
{'_id': 1936, 'published_count': 1}
{'_id': 1937, 'published_count': 1}
{'_id': 1939, 'published_count': 1}
{'_id': 1941, 'published_count': 2}
{'_id': 1944, 'published_count': 1}
{'_id': 1946, 'published_count': 1}
{'_id': 1948, 'published_count': 1}
{'_id': 1950, 'published_count': 1}
{'_id': 1951, 'published_count': 1}
{'_id': 1953, 'published_count': 1}
{'_id': 1954, 'published_count': 2}
{'_id': 1955, 'published_count': 1}
{'_id': 1958, 'published_count': 1}
{'_id': 1966, 'published_count': 2}
{'_id': 1976, 'published_cou

## [문제 5: 특정 사용자의 행동 유형 업데이트]

- **문제 설명**:
  특정 사용자의 행동 로그 중, 특정 날짜 이전의 "view" 행동을 "seen"으로 변경하고 싶습니다. 예를 들어, 사용자 ID가 1인 사용자의 2023년 4월 10일 이전의 모든 "view" 행동을 "seen"으로 변경하는 작업입니다.
- **쿼리 작성 목표**:
  사용자 ID가 1인 사용자의 2023년 4월 10일 이전의 "view" 행동을 "seen"으로 변경하는 MongoDB 업데이트 쿼리를 함수로 만들어 문제를 해결해 봅니다.


In [94]:
user_id = 2
base_time = datetime(2008, 4, 10)

update_user_actions_before_date(db, user_id, base_time, "view", "seen")

_match = {"$match": {"user_id": user_id, "timestamp": {"$lt": base_time}}}
_projection = {"$project": {"_id": 0, "action": 1, "timestamp": 1}}
_sort = {"$sort": {"timestamp": 1}}

_aggregate = [
    _match,
    _projection,
    _sort,
]

for e in db["user_actions"].aggregate(_aggregate):
    ### REVIEW - projection 단계에서 timestamp를 미리 문자열로 바꿀 수는 없을까?
    # print(e)
    print({"action": e["action"], "timestamp": e["timestamp"].strftime("%y-%m-%d")})

{'action': 'seen', 'timestamp': datetime.datetime(2003, 1, 19, 12, 26, 14, 947000)}


## 더 해볼 것들

- [Query on Embedded / Nested Documents](https://www.mongodb.com/docs/manual/tutorial/query-embedded-documents/)
- [Query for Null or Missing Fields](https://www.mongodb.com/docs/manual/tutorial/query-for-null-fields/) find에 다양한 조건들을 사용해보자
  - `"$ne"`
  - `"$type"`
  - `"$exists"`
